## Low-Tubal-Rank Autoregressive Tensor Completion Imputer (LATC-Tubal-imputer)

This notebook shows how to implement a LATC-Tubal imputer on some real-world large-scale data sets. To overcome the problem of missing values within multivariate time series data, this method takes into account both low-rank structure and time series regression. Meanwhile, to make the model scalable, we also integrate linear transform into the LATC model. For an in-depth discussion of LATC-Tubal-imputer, please see [1].

<div class="alert alert-block alert-info">
<font color="black">
<b>[1]</b> Xinyu Chen, Yixian Chen, Lijun Sun (2020). <b>Scalable low-rank autoregressive tensor learning for spatiotemporal traffic data imputation</b>. arXiv: 2008.03194. <a href="https://arxiv.org/abs/2008.03194" title="PDF"><b>[PDF]</b></a> <a href="https://doi.org/10.5281/zenodo.3939792" title="data"><b>[data]</b></a> 
</font>
</div>


In [1]:
import numpy as np
from numpy.linalg import inv as inv

### Define LATC-imputer kernel

We start by introducing some necessary functions that relies on `Numpy`.

<div class="alert alert-block alert-warning">
<ul>
<li><b><code>ten2mat</code>:</b> <font color="black">Unfold tensor as matrix by specifying mode.</font></li>
<li><b><code>mat2ten</code>:</b> <font color="black">Fold matrix as tensor by specifying dimension (i.e, tensor size) and mode.</font></li>
<li><b><code>svt</code>:</b> <font color="black">Implement the process of Singular Value Thresholding (SVT).</font></li>
</ul>
</div>

In [2]:
def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def mat2ten(mat, dim, mode):
    index = list()
    index.append(mode)
    for i in range(dim.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(dim[index]), order = 'F'), 0, mode)

In [3]:
def unitary_transform(tensor, Phi):
    return np.einsum('kt, ijk -> ijt', Phi, tensor)

def inv_unitary_transform(tensor, Phi):
    return np.einsum('kt, ijt -> ijk', Phi, tensor)

In [4]:
def tsvt(tensor, Phi, tau):
    dim = tensor.shape
    X = np.zeros(dim)
    tensor = unitary_transform(tensor, Phi)
    for t in range(dim[2]):
        u, s, v = np.linalg.svd(tensor[:, :, t], full_matrices = False)
        r = len(np.where(s > tau)[0])
        if r >= 1:
            s = s[: r]
            s[: r] = s[: r] - tau
            X[:, :, t] = u[:, : r] @ np.diag(s) @ v[: r, :]
    return inv_unitary_transform(X, Phi)

<div class="alert alert-block alert-warning">
<ul>
<li><b><code>compute_mape</code>:</b> <font color="black">Compute the value of Mean Absolute Percentage Error (MAPE).</font></li>
<li><b><code>compute_rmse</code>:</b> <font color="black">Compute the value of Root Mean Square Error (RMSE).</font></li>
</ul>
</div>

> Note that $$\mathrm{MAPE}=\frac{1}{n} \sum_{i=1}^{n} \frac{\left|y_{i}-\hat{y}_{i}\right|}{y_{i}} \times 100, \quad\mathrm{RMSE}=\sqrt{\frac{1}{n} \sum_{i=1}^{n}\left(y_{i}-\hat{y}_{i}\right)^{2}},$$ where $n$ is the total number of estimated values, and $y_i$ and $\hat{y}_i$ are the actual value and its estimation, respectively.

In [5]:
def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

The main idea behind LATC-imputer is to approximate partially observed data with both low-rank structure and time series dynamics. The following `imputer` kernel includes some necessary inputs:

<div class="alert alert-block alert-warning">
<ul>
<li><b><code>dense_tensor</code>:</b> <font color="black">This is an input which has the ground truth for validation. If this input is not available, you could use <code>dense_tensor = sparse_tensor.copy()</code> instead.</font></li>
<li><b><code>sparse_tensor</code>:</b> <font color="black">This is a partially observed tensor which has many missing entries.</font></li>
<li><b><code>time_lags</code>:</b> <font color="black">Time lags, e.g., <code>time_lags = np.array([1, 2, 3])</code>. </font></li>
<li><b><code>alpha</code>:</b> <font color="black">Weights for tensors' nuclear norm, e.g., <code>alpha = np.ones(3) / 3</code>. </font></li>
<li><b><code>rho</code>:</b> <font color="black">Learning rate for ADMM, e.g., <code>rho = 0.0005</code>. </font></li>
<li><b><code>lambda0</code>:</b> <font color="black">Weight for time series regressor, e.g., <code>lambda0 = 5 * rho</code>. If <code>lambda0 = 0</code>, then this imputer is actually a standard low-rank tensor completion (i.e., High-accuracy Low-Rank Tensor Completion, or HaLRTC).</font></li>
<li><b><code>epsilon</code>:</b> <font color="black">Stop criteria, e.g., <code>epsilon = 0.001</code>. </font></li>
<li><b><code>maxiter</code>:</b> <font color="black">Maximum iteration to stop algorithm, e.g., <code>maxiter = 50</code>. </font></li>
</ul>
</div>


In [6]:
def imputer(dense_tensor, sparse_tensor, time_lags, rho0, lambda0, epsilon, maxiter):
    """Low-Tubal-Rank Autoregressive Tensor Completion, LATC-Tubal-imputer."""
    dim = np.array(sparse_tensor.shape)
    dim_time = np.int(np.prod(dim) / dim[0])
    d = len(time_lags)
    max_lag = np.max(time_lags)
    sparse_mat = ten2mat(sparse_tensor, 0)
    pos_missing = np.where(sparse_mat == 0)
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    
    T = np.zeros(dim)                         # \boldsymbol{\mathcal{T}}
    Z = sparse_mat.copy()                     # \boldsymbol{Z}
    Z[pos_missing] = np.mean(sparse_mat[sparse_mat != 0])
    A = 0.001 * np.random.rand(dim[0], d)     # \boldsymbol{A}
    it = 0
    ind = np.zeros((d, dim_time - max_lag), dtype = np.int_)
    for i in range(d):
        ind[i, :] = np.arange(max_lag - time_lags[i], dim_time - time_lags[i])
    last_mat = sparse_mat.copy()
    snorm = np.linalg.norm(sparse_mat, 'fro')
    rho = rho0
    temp1 = ten2mat(mat2ten(Z, dim, 0), 2)
    _, Phi = np.linalg.eig(temp1 @ temp1.T)
    del temp1
    while True:
        rho = min(rho * 1.05, 1e5)
        X = tsvt(mat2ten(Z, dim, 0) - T / rho, Phi, 1 / rho)
        mat_hat = ten2mat(X, 0)
        mat0 = np.zeros((dim[0], dim_time - max_lag))
        temp2 = ten2mat(rho * X + T, 0)
        if lambda0 > 0:
            for m in range(dim[0]):
                Qm = mat_hat[m, ind].T
                A[m, :] = np.linalg.pinv(Qm) @ Z[m, max_lag :]
                mat0[m, :] = Qm @ A[m, :]
            Z[pos_missing] = np.append((temp2[:, : max_lag] / rho), (temp2[:, max_lag :] + lambda0 * mat0) 
                                       / (rho + lambda0), axis = 1)[pos_missing]
        else:
            Z[pos_missing] = temp2[pos_missing] / rho
        T = T + rho * (X - mat2ten(Z, dim, 0))
        tol = np.linalg.norm((mat_hat - last_mat), 'fro') / snorm
        last_mat = mat_hat.copy()
        it += 1
        if it % 10 == 0:
            temp1 = ten2mat(mat2ten(Z, dim, 0) - T / rho, 2)
            _, Phi = np.linalg.eig(temp1 @ temp1.T)
            del temp1
        if it % 1 == 0:
            print('Iter: {}'.format(it))
            print('Tolerance: {:.6}'.format(tol))
            var = dense_tensor[pos_test]
            var_hat = X[pos_test]
            print('MAPE: {:.6}'.format(compute_mape(var, var_hat)))
            print('RMSE: {:.6}'.format(compute_rmse(var, var_hat)))
            print()
        if (tol < epsilon) or (it >= maxiter):
            break

    print('Total iteration: {}'.format(it))
    print('Tolerance: {:.6}'.format(tol))
    var = dense_tensor[pos_test]
    var_hat = X[pos_test]
    print('Imputation MAPE: {:.6}'.format(compute_mape(var, var_hat)))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(var, var_hat)))
    print()
    
    return X

If you want to set parameters reasonably, please use this cross validation on your data set.

### California data - 4W

We generate **random missing (RM)** values on California traffic speed data set.

In [9]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 4 * 7)

missing_rate = 0.3

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

del data, random_tensor, binary_tensor

In [11]:
import time
start = time.time()
time_lags = np.array([1, 2, 3, 4, 5, 6, 286, 287, 288, 289, 290, 291])
rho = 1e-4
lambda0 = 0 * rho
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, time_lags, rho, lambda0, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.647327
MAPE: 0.0899981
RMSE: 6.28607

Iter: 2
Tolerance: 0.0648479
MAPE: 0.0619249
RMSE: 4.90082

Iter: 3
Tolerance: 0.0370936
MAPE: 0.0526588
RMSE: 4.45621

Iter: 4
Tolerance: 0.0265425
MAPE: 0.0470238
RMSE: 4.06033

Iter: 5
Tolerance: 0.0256821
MAPE: 0.0417275
RMSE: 3.67987

Iter: 6
Tolerance: 0.0200943
MAPE: 0.0385841
RMSE: 3.43764

Iter: 7
Tolerance: 0.0186521
MAPE: 0.0352242
RMSE: 3.18365

Iter: 8
Tolerance: 0.0167815
MAPE: 0.0325397
RMSE: 2.97991

Iter: 9
Tolerance: 0.0156531
MAPE: 0.0300217
RMSE: 2.78089

Iter: 10
Tolerance: 0.0140798
MAPE: 0.027988
RMSE: 2.61363

Iter: 11
Tolerance: 0.0315906
MAPE: 0.0318513
RMSE: 2.81248

Iter: 12
Tolerance: 0.0140204
MAPE: 0.028184
RMSE: 2.53781

Iter: 13
Tolerance: 0.0159258
MAPE: 0.0247291
RMSE: 2.29613

Iter: 14
Tolerance: 0.0105826
MAPE: 0.0233741
RMSE: 2.19051

Iter: 15
Tolerance: 0.00860577
MAPE: 0.0222413
RMSE: 2.09715

Iter: 16
Tolerance: 0.00820462
MAPE: 0.0212591
RMSE: 2.01389

Iter: 17
Tolerance: 0.00745984
MAP

In [12]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 4 * 7)

missing_rate = 0.7

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

del data, random_tensor, binary_tensor

In [13]:
import time
start = time.time()
time_lags = np.array([1, 2, 3, 4, 5, 6, 286, 287, 288, 289, 290, 291])
rho = 1e-4
lambda0 = 0 * rho
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, time_lags, rho, lambda0, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 1.49565
MAPE: 0.111673
RMSE: 7.6865

Iter: 2
Tolerance: 0.0617153
MAPE: 0.0962786
RMSE: 6.60298

Iter: 3
Tolerance: 0.0689141
MAPE: 0.0764408
RMSE: 5.57124

Iter: 4
Tolerance: 0.0610141
MAPE: 0.0647303
RMSE: 5.1603

Iter: 5
Tolerance: 0.0446548
MAPE: 0.0618997
RMSE: 5.09779

Iter: 6
Tolerance: 0.0329652
MAPE: 0.0598007
RMSE: 4.94375

Iter: 7
Tolerance: 0.0315494
MAPE: 0.0556912
RMSE: 4.63752

Iter: 8
Tolerance: 0.0337556
MAPE: 0.0503774
RMSE: 4.26924

Iter: 9
Tolerance: 0.0333318
MAPE: 0.0464125
RMSE: 3.98788

Iter: 10
Tolerance: 0.0290838
MAPE: 0.0446962
RMSE: 3.8367

Iter: 11
Tolerance: 0.0543425
MAPE: 0.0466334
RMSE: 3.89826

Iter: 12
Tolerance: 0.0225804
MAPE: 0.0469646
RMSE: 3.92097

Iter: 13
Tolerance: 0.0204167
MAPE: 0.043974
RMSE: 3.71667

Iter: 14
Tolerance: 0.0243385
MAPE: 0.0397729
RMSE: 3.4319

Iter: 15
Tolerance: 0.0247116
MAPE: 0.0364274
RMSE: 3.20929

Iter: 16
Tolerance: 0.0217448
MAPE: 0.034473
RMSE: 3.0748

Iter: 17
Tolerance: 0.018344
MAPE: 0.033078

We generate **non-random missing (NM)** values on Guangzhou traffic speed data set. Then, we conduct the imputation experiment.

In [14]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 4 * 7)

missing_rate = 0.3

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

del data, random_matrix, binary_tensor

In [15]:
import time
start = time.time()
time_lags = np.array([1, 2, 3, 4, 5, 6, 286, 287, 288, 289, 290, 291])
rho = 1e-4
lambda0 = 0 * rho
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, time_lags, rho, lambda0, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 1
Tolerance: 0.645668
MAPE: 0.0918986
RMSE: 6.39562

Iter: 2
Tolerance: 0.0640492
MAPE: 0.0655782
RMSE: 5.0786

Iter: 3
Tolerance: 0.0361203
MAPE: 0.057007
RMSE: 4.73378

Iter: 4
Tolerance: 0.0274561
MAPE: 0.0539009
RMSE: 4.57009

Iter: 5
Tolerance: 0.0216712
MAPE: 0.0531881
RMSE: 4.50298

Iter: 6
Tolerance: 0.0187843
MAPE: 0.0540346
RMSE: 4.49774

Iter: 7
Tolerance: 0.0170041
MAPE: 0.0552854
RMSE: 4.52424

Iter: 8
Tolerance: 0.0148231
MAPE: 0.0560778
RMSE: 4.54865

Iter: 9
Tolerance: 0.0130728
MAPE: 0.056419
RMSE: 4.56041

Iter: 10
Tolerance: 0.011685
MAPE: 0.0564941
RMSE: 4.5622

Iter: 11
Tolerance: 0.0280817
MAPE: 0.0570746
RMSE: 4.60072

Iter: 12
Tolerance: 0.0148322
MAPE: 0.0571865
RMSE: 4.60795

Iter: 13
Tolerance: 0.0113427
MAPE: 0.0572587
RMSE: 4.60819

Iter: 14
Tolerance: 0.00835246
MAPE: 0.0574437
RMSE: 4.61634

Iter: 15
Tolerance: 0.00746292
MAPE: 0.0575385
RMSE: 4.62012

Iter: 16
Tolerance: 0.00682469
MAPE: 0.057556
RMSE: 4.6197

Iter: 17
Tolerance: 0.00605841
MAPE: 0

In [16]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 4 * 7)

missing_rate = 0.7

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

del data, random_matrix, binary_tensor

In [17]:
import time
start = time.time()
time_lags = np.array([1, 2, 3, 4, 5, 6, 286, 287, 288, 289, 290, 291])
rho = 1e-4
lambda0 = 0 * rho
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, time_lags, rho, lambda0, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 1
Tolerance: 1.49254
MAPE: 0.111823
RMSE: 7.70025

Iter: 2
Tolerance: 0.0662397
MAPE: 0.0975341
RMSE: 6.70065

Iter: 3
Tolerance: 0.067056
MAPE: 0.083139
RMSE: 5.96412

Iter: 4
Tolerance: 0.0482453
MAPE: 0.0746097
RMSE: 5.61509

Iter: 5
Tolerance: 0.0308453
MAPE: 0.0698433
RMSE: 5.43514

Iter: 6
Tolerance: 0.0267958
MAPE: 0.0666876
RMSE: 5.30748

Iter: 7
Tolerance: 0.0236707
MAPE: 0.0646733
RMSE: 5.21599

Iter: 8
Tolerance: 0.0202557
MAPE: 0.0637341
RMSE: 5.15437

Iter: 9
Tolerance: 0.0185578
MAPE: 0.0638182
RMSE: 5.12322

Iter: 10
Tolerance: 0.0178718
MAPE: 0.064626
RMSE: 5.12181

Iter: 11
Tolerance: 0.0349142
MAPE: 0.0644829
RMSE: 5.04823

Iter: 12
Tolerance: 0.0164566
MAPE: 0.0652368
RMSE: 5.06785

Iter: 13
Tolerance: 0.0140252
MAPE: 0.0658471
RMSE: 5.08618

Iter: 14
Tolerance: 0.0123171
MAPE: 0.066265
RMSE: 5.09894

Iter: 15
Tolerance: 0.0107694
MAPE: 0.066522
RMSE: 5.10754

Iter: 16
Tolerance: 0.00976923
MAPE: 0.0666493
RMSE: 5.11219

Iter: 17
Tolerance: 0.0088423
MAPE: 0.06

### California data - 8W

We generate **random missing (RM)** values on California traffic speed data set.

In [18]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 8 * 7)

missing_rate = 0.3

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

del data, random_tensor, binary_tensor

We use `imputer` to fill in the missing entries and measure performance metrics on the ground truth.

In [19]:
import time
start = time.time()
time_lags = np.array([1, 2, 3, 4, 5, 6, 286, 287, 288, 289, 290, 291])
rho = 1e-4
lambda0 = 0 * rho
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, time_lags, rho, lambda0, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 1
Tolerance: 0.649777
MAPE: 0.0876833
RMSE: 6.14259

Iter: 2
Tolerance: 0.0661856
MAPE: 0.0585978
RMSE: 4.77865

Iter: 3
Tolerance: 0.0341726
MAPE: 0.0517565
RMSE: 4.44603

Iter: 4
Tolerance: 0.0234313
MAPE: 0.0470472
RMSE: 4.12706

Iter: 5
Tolerance: 0.0245962
MAPE: 0.0422044
RMSE: 3.76303

Iter: 6
Tolerance: 0.0211747
MAPE: 0.0390642
RMSE: 3.50142

Iter: 7
Tolerance: 0.0183585
MAPE: 0.0361345
RMSE: 3.27137

Iter: 8
Tolerance: 0.0175017
MAPE: 0.0332203
RMSE: 3.04554

Iter: 9
Tolerance: 0.0163321
MAPE: 0.0306397
RMSE: 2.83659

Iter: 10
Tolerance: 0.0144244
MAPE: 0.0286166
RMSE: 2.66649

Iter: 11
Tolerance: 0.0324494
MAPE: 0.0324851
RMSE: 2.86026

Iter: 12
Tolerance: 0.0143361
MAPE: 0.0287615
RMSE: 2.57955

Iter: 13
Tolerance: 0.0161566
MAPE: 0.0252091
RMSE: 2.33197

Iter: 14
Tolerance: 0.0108466
MAPE: 0.0237851
RMSE: 2.22074

Iter: 15
Tolerance: 0.00879176
MAPE: 0.0226284
RMSE: 2.12606

Iter: 16
Tolerance: 0.00819293
MAPE: 0.0216428
RMSE: 2.04332

Iter: 17
Tolerance: 0.00746881
M

In [20]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 8 * 7)

missing_rate = 0.7

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

del data, random_tensor, binary_tensor

In [21]:
import time
start = time.time()
time_lags = np.array([1, 2, 3, 4, 5, 6, 286, 287, 288, 289, 290, 291])
rho = 1e-4
lambda0 = 0 * rho
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, time_lags, rho, lambda0, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 1
Tolerance: 1.50177
MAPE: 0.114411
RMSE: 7.84667

Iter: 2
Tolerance: 0.0700399
MAPE: 0.0927179
RMSE: 6.39308

Iter: 3
Tolerance: 0.0751557
MAPE: 0.0707443
RMSE: 5.35096

Iter: 4
Tolerance: 0.0614989
MAPE: 0.0619116
RMSE: 5.10319

Iter: 5
Tolerance: 0.0413745
MAPE: 0.0615923
RMSE: 5.12944

Iter: 6
Tolerance: 0.0296792
MAPE: 0.059783
RMSE: 4.97614

Iter: 7
Tolerance: 0.0309106
MAPE: 0.0550474
RMSE: 4.64833

Iter: 8
Tolerance: 0.0332183
MAPE: 0.0500046
RMSE: 4.31709

Iter: 9
Tolerance: 0.0319974
MAPE: 0.0469406
RMSE: 4.07036

Iter: 10
Tolerance: 0.0283865
MAPE: 0.0456104
RMSE: 3.90896

Iter: 11
Tolerance: 0.0563782
MAPE: 0.0468761
RMSE: 3.9547

Iter: 12
Tolerance: 0.0251123
MAPE: 0.0476402
RMSE: 4.03345

Iter: 13
Tolerance: 0.0207673
MAPE: 0.0446923
RMSE: 3.83382

Iter: 14
Tolerance: 0.0257518
MAPE: 0.0401447
RMSE: 3.50902

Iter: 15
Tolerance: 0.0268865
MAPE: 0.0364423
RMSE: 3.24904

Iter: 16
Tolerance: 0.0236496
MAPE: 0.0345573
RMSE: 3.10821

Iter: 17
Tolerance: 0.0193111
MAPE: 0.

We generate **non-random missing (NM)** values on California traffic speed data set. Then, we conduct the imputation experiment.

In [22]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 8 * 7)

missing_rate = 0.3

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

del data, random_matrix, binary_tensor

In [23]:
import time
start = time.time()
time_lags = np.array([1, 2, 3, 4, 5, 6, 286, 287, 288, 289, 290, 291])
rho = 1e-4
lambda0 = 0 * rho
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, time_lags, rho, lambda0, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 1
Tolerance: 0.647614
MAPE: 0.0890179
RMSE: 6.22464

Iter: 2
Tolerance: 0.0655421
MAPE: 0.0611029
RMSE: 4.90966

Iter: 3
Tolerance: 0.0331578
MAPE: 0.0544135
RMSE: 4.64676

Iter: 4
Tolerance: 0.0257956
MAPE: 0.0517489
RMSE: 4.50791

Iter: 5
Tolerance: 0.0220123
MAPE: 0.0507498
RMSE: 4.42466

Iter: 6
Tolerance: 0.0187798
MAPE: 0.0516168
RMSE: 4.42067

Iter: 7
Tolerance: 0.0170084
MAPE: 0.0526028
RMSE: 4.43869

Iter: 8
Tolerance: 0.0154725
MAPE: 0.0531649
RMSE: 4.45171

Iter: 9
Tolerance: 0.0132569
MAPE: 0.053401
RMSE: 4.45866

Iter: 10
Tolerance: 0.0120932
MAPE: 0.0534321
RMSE: 4.45877

Iter: 11
Tolerance: 0.0293209
MAPE: 0.053942
RMSE: 4.49089

Iter: 12
Tolerance: 0.0144357
MAPE: 0.0539042
RMSE: 4.49156

Iter: 13
Tolerance: 0.0118832
MAPE: 0.0538054
RMSE: 4.48399

Iter: 14
Tolerance: 0.00868063
MAPE: 0.0539436
RMSE: 4.4906

Iter: 15
Tolerance: 0.00737546
MAPE: 0.0540124
RMSE: 4.49339

Iter: 16
Tolerance: 0.006888
MAPE: 0.0540092
RMSE: 4.49211

Iter: 17
Tolerance: 0.006331
MAPE: 0

In [24]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 8 * 7)

missing_rate = 0.7

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

del data, random_matrix, binary_tensor

In [25]:
import time
start = time.time()
time_lags = np.array([1, 2, 3, 4, 5, 6, 286, 287, 288, 289, 290, 291])
rho = 1e-4
lambda0 = 0 * rho
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, time_lags, rho, lambda0, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 1
Tolerance: 1.50012
MAPE: 0.114658
RMSE: 7.85768

Iter: 2
Tolerance: 0.0730121
MAPE: 0.0942427
RMSE: 6.50734

Iter: 3
Tolerance: 0.071667
MAPE: 0.0755545
RMSE: 5.58775

Iter: 4
Tolerance: 0.0520256
MAPE: 0.0660667
RMSE: 5.23676

Iter: 5
Tolerance: 0.0354722
MAPE: 0.0628432
RMSE: 5.14213

Iter: 6
Tolerance: 0.0269865
MAPE: 0.0607456
RMSE: 5.06694

Iter: 7
Tolerance: 0.0237192
MAPE: 0.0587474
RMSE: 4.97434

Iter: 8
Tolerance: 0.0230834
MAPE: 0.057515
RMSE: 4.89135

Iter: 9
Tolerance: 0.02257
MAPE: 0.0576788
RMSE: 4.84885

Iter: 10
Tolerance: 0.0201066
MAPE: 0.0589919
RMSE: 4.85231

Iter: 11
Tolerance: 0.0368248
MAPE: 0.0593495
RMSE: 4.79108

Iter: 12
Tolerance: 0.0177441
MAPE: 0.0604448
RMSE: 4.82626

Iter: 13
Tolerance: 0.0147386
MAPE: 0.0610166
RMSE: 4.84504

Iter: 14
Tolerance: 0.0131677
MAPE: 0.06122
RMSE: 4.84994

Iter: 15
Tolerance: 0.0119143
MAPE: 0.0612398
RMSE: 4.84985

Iter: 16
Tolerance: 0.0106928
MAPE: 0.061173
RMSE: 4.84836

Iter: 17
Tolerance: 0.00953009
MAPE: 0.0610

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>